# 08 - Context Engineering for Reliable Agents

This notebook explores the principles of **Context Engineering** and how we apply them to make our Polymarket agents more reliable.

## What is Context Engineering?

Context engineering is providing the right information and tools in the right format so the LLM can accomplish a task. Most agent failures are caused by passing the "wrong" context rather than model capability issues.

We categorize context into three sources:
1. **Runtime Context**: Static configuration (User role, Environment).
2. **State**: Short-term conversation memory (Message counts, auth status).
3. **Store**: Long-term persistent knowledge (Market stats, user preferences).

## The Context Manager

In this project, we use a `ContextManager` to orchestrate these sources.

In [ ]:
from polymarket_agents.utils.context import ContextManager, RuntimeContext
from pprint import pprint

# 1. Setup Runtime Context
runtime = RuntimeContext(
    user_id="analyst_01",
    user_role="senior_analyst",
    deployment_env="production"
)

# 2. Initialize Context Manager
cm = ContextManager(runtime=runtime)

# 3. View the engineered context block
print("Engineered Context for LLM:")
print(cm.get_model_context())

## Dynamic State Triggers

We can update the context dynamically based on conversation state. For example, if the user authenticates, the agent's behavior and available information should change.

In [ ]:
# Simulate a state change (e.g., user logged in)
cm.update_state({"authenticated": True, "messages_count": 5})

print("Context after Authentication:")
print(cm.get_model_context())

## Context-Aware Agent Creation

Our agent factory `create_polymarket_agent` now accepts a `ContextManager` to automatically inject this structured data into the system prompt.

In [ ]:
from polymarket_agents.langchain.agent import create_polymarket_agent

# Create an agent with the context manager
agent = create_polymarket_agent(
    model="gpt-4o-mini",
    context_manager=cm,
    verbose=False
)

print("Agent successfully created with dynamic context injection.")

## Benefits of this Approach

1. **Hygiene**: Instead of dumping thousands of lines of raw memory, we provide structured summaries.
2. **Safety**: By injecting the `deployment_env`, the agent knows when it needs to be extra careful with execution tools.
3. **Consistency**: The LLM always has a clear view of its role and the system's current state (e.g., how many markets it has access to).